Evin McDonald, Milan Patel, Teja Sirigina, Yolanda Qiu, David Gong

In [ ]:
!pip install selenium

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 46.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.2/400.2 kB 34.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.9 MB/s eta 0:00:00


In [ ]:
# Import necessary libraries
from selenium import webdriver
from selenium.webdriver.common.by import By


In [ ]:
from bs4 import BeautifulSoup
import pandas as pd

In [ ]:
# Set Chrome options for headless browsing
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

# Initialize the Chrome webdriver with the specified options
driver = webdriver.Chrome(options=chrome_options)

# Create an empty DataFrame to store scraped data
comments = pd.DataFrame(columns=['Product_name', 'Product_review', 'User_rating'])

# Base URL
base_url = "https://www.beeradvocate.com"
# URLs to scrape
urls_to_scrape = ["https://www.beeradvocate.com/beer/top-rated/",
                   "https://www.beeradvocate.com/beer/popular/"]

# Loop through URLs to scrape data
for url in urls_to_scrape:
    driver.get(url)
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    beer_links = [link['href'] for index, link in enumerate(soup.select('td a[href^="/beer/profile/"]')) if index % 2 == 0]

    for beer_link in beer_links[:250]:
        driver.get(base_url + beer_link)
        beer_soup = BeautifulSoup(driver.page_source, 'html.parser')

        product_name = beer_soup.h1.contents[0].strip()
        reviews = driver.find_elements(By.XPATH, '//*[@id="rating_fullview_content_2"]')

        for idx, review in enumerate(reviews):
            if idx >= 20:
                break
            try:
                user_rating = review.find_element(By.XPATH, './/span[2]').text
                product_review = review.find_element(By.XPATH, './/div').text
            except:
                user_rating = review.find_element(By.XPATH, './/span[1]/b[1]').text
                product_review = ""

            # Append data to the dataset
            comments = comments.append({
                'Product_name': product_name,
                'User_rating': user_rating,
                'Product_review': product_review
            }, ignore_index=True)


# Close the Chrome webdriver
driver.quit()



In [ ]:
comments

NameError: ignored

In [ ]:
comments = pd.read_csv('popular.csv')


comments = comments.dropna()
comments

,Unnamed: 0,Product_name,Product_review,User_rating
2,2,Kentucky Brunch Brand Stout,"Amazing stout, expensive but worth the price!",5.00
5,5,Kentucky Brunch Brand Stout,This beer meet and exceeded all the unreal hyp...,5.00
7,7,Kentucky Brunch Brand Stout,Let it sit and warm to the room a little ... u...,5.00
8,8,Kentucky Brunch Brand Stout,"A small pour, ebony dark, no real head. Intens...",4.67
15,15,Kentucky Brunch Brand Stout,"Vintage 2022, served on tap at Toppling Goliat...",5.00
...,...,...,...,...
16830,16830,Konstantin,A very easy pour was needed to keep the head u...,4.24
16834,16834,Konstantin,Pours a dark reddish color with a small foamy ...,4.00
16841,16841,Konstantin,16 oz can picked up at the brewery\n\npours a ...,4.12
16847,16847,Blended With Lumber #1,Pours a dark brown color with a minimal head t...,4.25


We used an extensive dataset encompassing over 16,000 reviews. Subsequently, all entries lacking written reviews were systematically eliminated, resulting in the retention of 5,931 data rows.


In [ ]:
# CODE FOR (B)
import nltk
from nltk.corpus import stopwords
from collections import Counter
import re
nltk.download('punkt')
nltk.download('stopwords')

#nltk.download('stopwords')

# Prepare reviews data
reviews = comments['Product_review'].tolist()
all_reviews = " ".join(reviews).lower()  # Convert all reviews into a single string and make it lowercase

# Tokenize and remove stopwords
words = nltk.word_tokenize(all_reviews)
filtered_words = [word for word in words if word not in stopwords.words('english') and len(word) > 2]

# Count words
word_counts = Counter(filtered_words)

# View top 50 words to decide on attributes
top_words = word_counts.most_common(50)
print(top_words)



# Extract products based on user's chosen attributes
chosen_attributes = ['sweet', 'dark', 'malt']

def recommend_beer_based_on_attributes(attribute_list, df):
    recommended_beers = set()
    for attribute in attribute_list:
        subset = df[df['Product_review'].str.contains(attribute, case=False, na=False)]
        recommended_beers.update(subset['Product_name'].unique())

    # Return top 3 recommended beers
    return list(recommended_beers)[:3]

recommended_beers = recommend_beer_based_on_attributes(chosen_attributes, comments)
print(recommended_beers)

chosen_attributesdf = pd.DataFrame({'Attributes': chosen_attributes})
chosen_attributesdf.to_csv("attributes.csv")
from google.colab import files
files.download("attributes.csv")


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


[('beer', 4023), ('head', 3493), ('taste', 2937), ('dark', 2503), ('light', 1989), ('good', 1929), ('malt', 1923), ('sweet', 1877), ('like', 1814), ('chocolate', 1744), ('nice', 1725), ('one', 1712), ('carbonation', 1694), ('aroma', 1683), ('notes', 1654), ('finish', 1597), ('medium', 1584), ('body', 1541), ('pours', 1527), ('bottle', 1498), ('overall', 1459), ('lacing', 1384), ('flavor', 1368), ('coffee', 1355), ('brown', 1350), ('nose', 1348), ('bit', 1318), ('well', 1301), ('glass', 1260), ('white', 1242), ('caramel', 1186), ('mouthfeel', 1150), ('hops', 1109), ('little', 1101), ('bitterness', 1088), ('smooth', 1081), ('smell', 1077), ('feel', 1077), ('color', 1077), ("n't", 1045), ('great', 1022), ('fruit', 1020), ('bitter', 999), ('citrus', 989), ('poured', 984), ('black', 967), ('orange', 947), ('much', 877), ('really', 872), ('dry', 866)]
['Notorious Triple IPA', 'Lagunitas Sucks', 'Kentucky Brunch Brand Stout']


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
attributes = pd.read_csv('attributes.csv')
attributes

,Unnamed: 0,Attributes
0,0,sweet
1,1,dark
2,2,malt


We conducted a word frequency analysis on the dataset, yielding a sorted list of high-frequency words. From this list, we selected words that customers commonly use to define their preferred beer. For this illustrative example, the chosen descriptors were "Dark," "Sweet," and "Malt."

In [ ]:
# CODE FOR (C)


from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

product_attributes = list(attributes["Attributes"])
count_vectorizer = CountVectorizer(stop_words='english', lowercase=True)
count_matrix = count_vectorizer.fit_transform(comments['Product_review'])

# Convert the list of words to a numerical vector
word_vector = count_vectorizer.transform([' '.join(product_attributes)])

# Compute cosine similarity between each review and the list of words
similarities = cosine_similarity(count_matrix, word_vector)

# Create a DataFrame to store the results
result_df = pd.DataFrame({
    'product_name': comments['Product_name'],
    'review': comments['Product_review'],
    'cosine_similarity': similarities.flatten()
})


# Sort by similarity score
sorted_results = result_df.sort_values(by="cosine_similarity", ascending=False)
result_df
print(sorted_results)


                                  product_name  \
10968                                 Ten FIDY   
5448                                  Ten FIDY   
6524   Dragon's Milk Bourbon Barrel-Aged Stout   
4994                            Stickee Monkee   
15695  Dragon's Milk Bourbon Barrel-Aged Stout   
...                                        ...   
8289                               Shiner Bock   
8290                               Shiner Bock   
8294                               Shiner Bock   
8299                               Shiner Bock   
9372                                 Damnation   

                                                  review  cosine_similarity  
10968  Pours black as night. Dark foam. Sweet malt ar...            0.58554  
5448   Pours black as night. Dark foam. Sweet malt ar...            0.58554  
6524                                    Too sweet for me            0.57735  
4994                                   Sweet. Too sweet.            0.57735  
15695    

We computed the cosine similarity scores for each beer in the dataset.


In [ ]:
  # CODE FOR (D)
import nltk
nltk.download('vader_lexicon')
from nltk.sentiment import SentimentIntensityAnalyzer

sia = SentimentIntensityAnalyzer()

# Add sentiment scores to the DataFrame
result_df['compound_sentiment'] = result_df['review'].apply(lambda x: sia.polarity_scores(str(x))['compound'])

# Convert compound sentiment scores to sentiment categories
result_df['sentiment_category'] = result_df['compound_sentiment'].apply(lambda score: 'positive' if score > 0.05 else ('neutral' if -0.05 < score < 0.05 else 'negative'))

#print(result_df[['product_name', 'review', 'sentiment_category', 'compound_sentiment']])
result_df


[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


,product_name,review,cosine_similarity,compound_sentiment,sentiment_category
2,Kentucky Brunch Brand Stout,"Amazing stout, expensive but worth the price!",0.000000,0.6588,positive
5,Kentucky Brunch Brand Stout,This beer meet and exceeded all the unreal hyp...,0.000000,0.6239,positive
7,Kentucky Brunch Brand Stout,Let it sit and warm to the room a little ... u...,0.000000,0.3446,positive
8,Kentucky Brunch Brand Stout,"A small pour, ebony dark, no real head. Intens...",0.165900,0.9275,positive
15,Kentucky Brunch Brand Stout,"Vintage 2022, served on tap at Toppling Goliat...",0.067574,0.9075,positive
...,...,...,...,...,...
16830,Konstantin,A very easy pour was needed to keep the head u...,0.259238,0.9570,positive
16834,Konstantin,Pours a dark reddish color with a small foamy ...,0.132453,0.0000,neutral
16841,Konstantin,16 oz can picked up at the brewery\n\npours a ...,0.226455,0.6494,positive
16847,Blended With Lumber #1,Pours a dark brown color with a minimal head t...,0.246183,0.7845,positive


Then we conducted sentiment analysis on the dataset utilizing the Vader sentiment analysis tool.


In [ ]:
# CODE FOR (E)
# Combine Similarity and Sentiment Scores
similarity_weight = 0.5  # Adjust the weighting factors as needed
sentiment_weight = 0.5

result_df["combined_scores"] = (result_df['cosine_similarity'] * similarity_weight) + (result_df['compound_sentiment'] * sentiment_weight)

#result_df["combined_score": combined_scores.flatten()]
# Sort by combined score
sorted_results = result_df.sort_values(by="combined_scores", ascending=False)
print(sorted_results)
result_df

                                          product_name  \
7147                                              Yeti   
13287                                             Yeti   
16061                        Tucher Nürnberger Rotbier   
14705                                    Old Viscosity   
13354                        Weihenstephaner Korbinian   
...                                                ...   
12621                                       Alpha King   
6261                                        Alpha King   
13385                                      Pumpkin Ale   
12364                                      Oude Gueuze   
13647  Abbaye Des Rocs Grand Cru Belgian Special Brown   

                                                  review  cosine_similarity  \
7147   Opaque black color with some ruby highlights a...           0.560772   
13287  Opaque black color with some ruby highlights a...           0.560772   
16061  Pours a dark hazy brown color with a generous ...          

,product_name,review,cosine_similarity,compound_sentiment,sentiment_category,combined_scores
2,Kentucky Brunch Brand Stout,"Amazing stout, expensive but worth the price!",0.000000,0.6588,positive,0.329400
5,Kentucky Brunch Brand Stout,This beer meet and exceeded all the unreal hyp...,0.000000,0.6239,positive,0.311950
7,Kentucky Brunch Brand Stout,Let it sit and warm to the room a little ... u...,0.000000,0.3446,positive,0.172300
8,Kentucky Brunch Brand Stout,"A small pour, ebony dark, no real head. Intens...",0.165900,0.9275,positive,0.546700
15,Kentucky Brunch Brand Stout,"Vintage 2022, served on tap at Toppling Goliat...",0.067574,0.9075,positive,0.487537
...,...,...,...,...,...,...
16830,Konstantin,A very easy pour was needed to keep the head u...,0.259238,0.9570,positive,0.608119
16834,Konstantin,Pours a dark reddish color with a small foamy ...,0.132453,0.0000,neutral,0.066227
16841,Konstantin,16 oz can picked up at the brewery\n\npours a ...,0.226455,0.6494,positive,0.437928
16847,Blended With Lumber #1,Pours a dark brown color with a minimal head t...,0.246183,0.7845,positive,0.515341


In [ ]:
#Recommendation
first_3_product_namesbow = sorted_results['product_name'].unique()[:3]
print(first_3_product_namesbow)

['Yeti' 'Tucher Nürnberger Rotbier' 'Old Viscosity']


We combined similarity and sentiment scores by assigning both to a weight of 50%. To create our evaluation score, we multiplied the cosine similarity by the similarity weight and added it to the product of the compound sentiment score and the weight we assigned to it. After evaluating the combined scores, we recommend Yeti, Tucher Nürnberger Rotbier, and Old Viscosity as these three had the highest combined scores.


In [ ]:
#Code for (F)
import spacy
!python -m spacy download en_core_web_md
nlp = spacy.load('en_core_web_md')

2023-10-02 04:34:59.856911: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 MB 15.6 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')


In [ ]:
product_attributes_str = ' '.join(attributes["Attributes"])
attr = nlp(product_attributes_str)


#wordem_df['word_embedding_similarity'] = nlp(comments['Product_review']).similiarity(attr)
result_df['word_embedding_similarity'] = result_df['review'].apply(lambda review: nlp(review).similarity(nlp(attr)))
#similarity_scores.append(attr.similarity(rev))




<ipython-input-11-8265606307c0>:6: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  result_df['word_embedding_similarity'] = result_df['review'].apply(lambda review: nlp(review).similarity(nlp(attr)))


In [ ]:
print(attr)

sweet dark malt


In [ ]:
result_df

,product_name,review,cosine_similarity,compound_sentiment,sentiment_category,combined_scores,word_embedding_similarity
2,Kentucky Brunch Brand Stout,"Amazing stout, expensive but worth the price!",0.000000,0.6588,positive,0.329400,0.350086
5,Kentucky Brunch Brand Stout,This beer meet and exceeded all the unreal hyp...,0.000000,0.6239,positive,0.311950,0.393807
7,Kentucky Brunch Brand Stout,Let it sit and warm to the room a little ... u...,0.000000,0.3446,positive,0.172300,0.423204
8,Kentucky Brunch Brand Stout,"A small pour, ebony dark, no real head. Intens...",0.165900,0.9275,positive,0.546700,0.384170
15,Kentucky Brunch Brand Stout,"Vintage 2022, served on tap at Toppling Goliat...",0.067574,0.9075,positive,0.487537,0.366815
...,...,...,...,...,...,...,...
16830,Konstantin,A very easy pour was needed to keep the head u...,0.259238,0.9570,positive,0.608119,0.424300
16834,Konstantin,Pours a dark reddish color with a small foamy ...,0.132453,0.0000,neutral,0.066227,0.527102
16841,Konstantin,16 oz can picked up at the brewery\n\npours a ...,0.226455,0.6494,positive,0.437928,0.506263
16847,Blended With Lumber #1,Pours a dark brown color with a minimal head t...,0.246183,0.7845,positive,0.515341,0.525508


In [ ]:
result_df["combined_scoreswe"] = (result_df['word_embedding_similarity'] * similarity_weight) + (result_df['compound_sentiment'] * sentiment_weight)
result_df

,product_name,review,cosine_similarity,compound_sentiment,sentiment_category,combined_scores,word_embedding_similarity,combined_scoreswe
2,Kentucky Brunch Brand Stout,"Amazing stout, expensive but worth the price!",0.000000,0.6588,positive,0.329400,0.350086,0.504443
5,Kentucky Brunch Brand Stout,This beer meet and exceeded all the unreal hyp...,0.000000,0.6239,positive,0.311950,0.393807,0.508854
7,Kentucky Brunch Brand Stout,Let it sit and warm to the room a little ... u...,0.000000,0.3446,positive,0.172300,0.423204,0.383902
8,Kentucky Brunch Brand Stout,"A small pour, ebony dark, no real head. Intens...",0.165900,0.9275,positive,0.546700,0.384170,0.655835
15,Kentucky Brunch Brand Stout,"Vintage 2022, served on tap at Toppling Goliat...",0.067574,0.9075,positive,0.487537,0.366815,0.637158
...,...,...,...,...,...,...,...,...
16830,Konstantin,A very easy pour was needed to keep the head u...,0.259238,0.9570,positive,0.608119,0.424300,0.690650
16834,Konstantin,Pours a dark reddish color with a small foamy ...,0.132453,0.0000,neutral,0.066227,0.527102,0.263551
16841,Konstantin,16 oz can picked up at the brewery\n\npours a ...,0.226455,0.6494,positive,0.437928,0.506263,0.577832
16847,Blended With Lumber #1,Pours a dark brown color with a minimal head t...,0.246183,0.7845,positive,0.515341,0.525508,0.655004


In [ ]:
sorted_resultswe = result_df.sort_values(by="combined_scoreswe", ascending=False)
print(sorted_resultswe)


                                 product_name  \
2626                               Parabajava   
9577                    Wisconsin Belgian Red   
11017                   Wisconsin Belgian Red   
16817                      V.S.O. Gravedigger   
12331                          Fantôme Saison   
...                                       ...   
3949                             Double Stack   
10701  Expedition Stout - Bourbon Barrel-Aged   
12258                        Expedition Stout   
6598                         Expedition Stout   
16116            Dragon's Milk - Crimson Keep   

                                                  review  cosine_similarity  \
2626   Look - love the thick blackness with a nice th...           0.340503   
9577       great cherr ale slightly sweet great drinking           0.192450   
11017      great cherr ale slightly sweet great drinking           0.192450   
16817  Look - Black and thick with a nice tan head an...           0.000000   
12331  I am so h

In [ ]:
#Recommendation
first_3_product_nameswe = sorted_resultswe['product_name'].unique()[:3]
print(first_3_product_nameswe)

['Parabajava' 'Wisconsin Belgian Red' 'V.S.O. Gravedigger']


For BoW, it represents text as a vector where each element corresponds to the frequency (or presence) of a word from the vocabulary in the text. It does not capture semantic meaning. Words with similar meanings but different surface forms are treated as distinct. This can lead to very high-dimensional vectors, especially with large vocabularies. Therefore, BoW is more suitable for scenarios where exact word matches are more important, or when computational resources are limited. For Word Embeddings, it represents words as dense vectors, typically of fixed size, which requires training, but pretrained models on vast corpora are available. Each vector captures semantic relationships between words, where similar words or synonyms have vectors that are close in the vector space. We can capture these synonyms as they often have similar vectors; in this case, word embedding is ideal for scenarios where semantic matching is essential, like content-based recommendations, sentiment analysis. To see the combined score, we combined similarity and sentiment scores by assigning both scores to a weight of 50% as we did for the cosine similarity using bag of words, thereby, for word embedding, we have 3 different recommendation than from the bag of words, which are 'Parabajava', 'Wisconsin Belgian Red', and  'V.S.O. Gravedigger'. We then calculate the percentage of the reviews on each of the three with the top 3 attributes for both BoW and word embeddings.

In [ ]:
percentage_mentions = []
all_product_namesbow = []
for product_name in first_3_product_namesbow:
    product_reviews = result_df[result_df['product_name'] == product_name]['review']

    mentions = product_reviews.apply(lambda review: any(word in review.lower() for word in product_attributes)).sum()

    total_reviews = len(product_reviews)
    percentage = (mentions / total_reviews) * 100 if total_reviews > 0 else 0
    percentage_mentions.append(percentage)


    # Append the current product name to the list
    all_product_namesbow.append(product_name)
# Create a new DataFrame to store the results
percentagesdfbow = pd.DataFrame({
    'Product_name': all_product_namesbow,
    'Percentage_mentions': percentage_mentions
})
percentagesdfbow

,Product_name,Percentage_mentions
0,Yeti,100.000000
1,Tucher Nürnberger Rotbier,100.000000
2,Old Viscosity,83.333333


In [ ]:
percentage_mentions = []
all_product_nameswe = []
for product_name in first_3_product_nameswe:
    product_reviews = result_df[result_df['product_name'] == product_name]['review']

    mentions = product_reviews.apply(lambda review: any(word in review.lower() for word in product_attributes)).sum()

    total_reviews = len(product_reviews)
    percentage = (mentions / total_reviews) * 100 if total_reviews > 0 else 0
    percentage_mentions.append(percentage)


    # Append the current product name to the list
    all_product_nameswe.append(product_name)
# Create a new DataFrame to store the results
percentagesdfwe = pd.DataFrame({
    'Product_name': all_product_nameswe,
    'Percentage_mentions': percentage_mentions
})
percentagesdfwe

,Product_name,Percentage_mentions
0,Parabajava,76.923077
1,Wisconsin Belgian Red,66.666667
2,V.S.O. Gravedigger,71.428571


We see that for BoWs there are 100% mention for Yeti and Tucher Nürnberger Rotbier, and 88% of the Old Viscosity indicating a direct mention of the attributes. A percentage of 76.9 for 'Parabajava', and 66.6 'Wisconsin Belgian Red', and 71.4 for 'V.S.O. Gravedigger’, indicating the reviews mentioning preferred attributes with word embedding signifies a meaningful portion of the collected reviews that are closely related to the specific attributes of interest. Instead of the direct mention, it would also capture the semantic meaning for each of the attributes.  In practical terms, this means that the reviews that have close meaning to the attributes are also captured, and more reviews under consideration align well with the attributes that we have defined as preferred. These reviews are likely to contain content or descriptions that match the desired characteristics or features of the product.



This significance implies that a substantial portion of customer feedback, as represented by these reviews, directly addresses the aspects that are particularly important to your analysis, such as product quality, taste, or any of the other attributes specified. This information can be invaluable for decision-making, as it allows you to focus on a subset of reviews that are highly relevant to your specific criteria, making it easier to gain insights into customer satisfaction or to make informed recommendations.


In [ ]:
#Code for (G)
sortcomments = comments.sort_values(by="User_rating", ascending=False)
first_3_product_namesrat = sortcomments['Product_name'].unique()[:3]
print(first_3_product_namesrat)

['Kentucky Brunch Brand Stout' 'Atrial Rubicite' 'Péché Mortel']


In [ ]:
percentage_mentions = []
all_product_namesrat = []
for product_name in first_3_product_namesrat:
    product_reviews = result_df[result_df['product_name'] == product_name]['review']

    mentions = product_reviews.apply(lambda review: any(word in review.lower() for word in product_attributes)).sum()

    total_reviews = len(product_reviews)
    percentage = (mentions / total_reviews) * 100 if total_reviews > 0 else 0
    percentage_mentions.append(percentage)


    # Append the current product name to the list
    all_product_namesrat.append(product_name)
# Create a new DataFrame to store the results
percentagesdfrat = pd.DataFrame({
    'Product_name': all_product_namesrat,
    'Percentage_mentions': percentage_mentions
})
percentagesdfrat

,Product_name,Percentage_mentions
0,Kentucky Brunch Brand Stout,40.000000
1,Atrial Rubicite,14.285714
2,Péché Mortel,75.000000


 Ignoring the sentiment scores and similarity scores, and simply recommending the products with the highest ratings can be problematic for a few reasons. These products might have received high ratings for reasons unrelated to the attributes the user is looking for; for example, they could have high ratings due to factors like brand reputation, marketing, or other characteristics that may not align with the user's preferences. The recommended products might not meet the user's requirements for specific attributes (in this case, "Dark," "Sweet," and "Malt") because overall ratings do not provide information about these attributes.
The recommended products will always be the same no matter what attributes the user picks, so the results will never be tailored. For example, the highest rated products are Kentucky Brunch Brand Stout, Atrial Rubicite, and Péché Mortel. These will always be the products recommended by this method. With the current set of attributes, Kentucky Brunch Brand Stout has 40 percent mentions, Atrial Rubicite has 14.2 percent mentions, and Péché Mortel has 75 percent mentions. As you can see, the first two recommendations have a very low percentage of mentions and these 3 mention percentages will fluctuate depending on the attributes but the recommended beers will always be those 3.


In [ ]:
#Code for (H)
top4attributes = ['sweet', 'dark', 'malt','aroma']
beers = ['Kentucky Brunch Brand Stout', 'Goose IPA', 'Maximus', 'Impermanence', 'Darkness', 'Double Sunshine', 'Flower Power', 'Bourbon County Brand Stout', 'Abt 12', 'Troegenator' ]
lift_values = []

for beer in beers:
    lift = 0  # Initialize lift value
    for attribute in top4attributes:
        # Calculate Support(attribute, beer), Support(attribute), and Support(beer) here
        support_attribute_beer = len(comments[(comments['Product_name'] == beer) & comments['Product_review'].str.contains(attribute)]) / len(comments)
        support_attribute = len(comments[comments['Product_review'].str.contains(attribute)]) / len(comments)
        support_beer = len(comments[comments['Product_name'] == beer]) / len(comments)

        # Calculate Lift(attribute, beer)
        if support_attribute != 0 and support_beer != 0:
            lift += (support_attribute_beer / (support_attribute * support_beer))

    lift_values.append({'Beer': beer, 'Lift': lift})

# Find the most similar beer based on lift values
lift_df = pd.DataFrame(lift_values)
print(lift_df)
reference_beer = 'Maximus'
lift_df['Absolute_Difference'] = abs(lift_df['Lift'] - lift_df[lift_df['Beer'] == reference_beer]['Lift'].values[0])

# Find the beer with the closest lift value to the reference beer
closest_beer = lift_df[lift_df['Beer'] != reference_beer].sort_values(by='Absolute_Difference').iloc[0]

print(f"The beer closest in lift value to '{reference_beer}' is '{closest_beer['Beer']}' with a lift value of {closest_beer['Lift']:.2f}")

                          Beer      Lift
0  Kentucky Brunch Brand Stout  3.031908
1                    Goose IPA  2.600454
2                      Maximus  2.392454
3                 Impermanence  8.070771
4                     Darkness  7.493729
5              Double Sunshine  5.660808
6                 Flower Power  3.882413
7   Bourbon County Brand Stout  3.696648
8                       Abt 12  3.394881
9                  Troegenator  4.951219
The beer closest in lift value to 'Maximus' is 'Goose IPA' with a lift value of 2.60


We aimed to determine the most similar beer to a reference beer, Maximus, based on the association between specific attributes and a list of beers. The code works by defining a set of top 4 attributes ( "Dark," "Sweet," “Aroma” and "Malt") and a list of beers to evaluate. The code then calculates lift values for each attribute-beer pair by considering the proportion of reviews mentioning the attribute, the beer, and both together. For each beer, the code iterates through the attributes, computes lift values, and stores the results in a dataframe. Then it identifies the most similar beer to the reference beer by subtracting the lift of each beer from the lift of the reference beer and getting the absolute value. This shows which one is closest by finding the smallest absolute value. Finally, after printing all the lift values for all beers it picks the most similar beer, along with its corresponding lift value. In this example with these attributes selected, “the most similar beer to Maximus based on attribute association is 'Goose IPA' with a lift value of 2.60.”
